# ATTENTION

# ON MODIFIE DES VRAIES DONNEES USAGERS

# A MANIPULER AVEC BEAUCOUP DE PRECAUTIONS

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

from unidecode import unidecode

import requests
import json

import datetime

from kiblib.utils.db import DbConn
from kiblib.utils.code2libelle import Code2Libelle
from kiblib.adherent import Adherent

## Définition et test fonction de modification

In [2]:
def mod_borrower(userid, data):
    api_url = "http://cataloguekoha.ntrbx.local/cgi-bin/koha/rest.pl/user"
    url = f"{api_url}/{userid}"
    data = json.dumps(data)
    data2mod = f"data={data}"
    print(data2mod)
    response = requests.put(url, data=data2mod)
    print(f"{userid} : {response.content}")

In [3]:
data = {'city':'HAZEBROUCK'}
userid = 'fpichenot'
mod_borrower(userid, data)

data={"city": "HAZEBROUCK"}
fpichenot : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'


## Import des adhérents de la base de production

In [38]:
adherents = pd.read_json("http://cataloguekoha.ntrbx.local/cgi-bin/koha/rest.pl/user")

In [39]:
adherents.columns

Index(['surname', 'altcontactsurname', 'altcontactzipcode', 'phone', 'B_state',
       'B_country', 'fax', 'country', 'sms_provider_id', 'emailpro',
       'streetnumber', 'mobile', 'contactnote', 'altcontactfirstname',
       'initials', 'lastseen', 'sort2', 'B_streetnumber', 'state',
       'altcontactphone', 'contacttitle', 'privacy', 'autorenew_checkouts',
       'categorycode', 'altcontactstate', 'phonepro', 'flags',
       'primary_contact_method', 'lang', 'firstname',
       'privacy_guarantor_fines', 'contactname', 'opacnote', 'login_attempts',
       'email', 'title', 'B_email', 'debarred', 'B_city', 'relationship',
       'B_streettype', 'anonymized', 'altcontactaddress1', 'cardnumber',
       'lost', 'contactfirstname', 'userid', 'B_zipcode', 'sex',
       'checkprevcheckout', 'dateofbirth', 'dateenrolled', 'B_address2',
       'sort1', 'altcontactaddress3', 'gonenoaddress', 'borrowernotes',
       'borrowernumber', 'address2', 'updated_on', 'altcontactcountry',
       'pass

## Correction des villes

In [40]:
adherents = adherents.rename(columns={"cardnumber":"numéro de carte",
                              "surname":"nom",
                              "firstname":"prénom",
                              "title": "Madame ou Monsieur",
                              "othernames":"nom de l'épouse",
                              "address":"adresse",
                              "address2":"complément d'adresse",
                              "city":"ville",
                              "zipcode":"code postal",
                              "country":"pays",
                              "phone":"numéro de téléphone fixe",
                              "mobile":"numéro de téléphone portable",
                              "B_address":"date de l'appel téléphonique",
                              "B_address2":"résultat de l'appel",
                              "B_city":"date demande création tiers",
                              "B_state":"numéro du titre de recette",
                              "B_zipcode":"numéro de tiers GF",
                              "B_country":"date création titre de recette",
                              "dateofbirth":"date de naissance",
                              "branchcode":"site de rattachement",
                              "categorycode":"type de carte",
                              "dateenrolled":"date d'inscription",
                              "dateexpiry":"date d'expiration",
                              "date_renewed":"date de renouvellement",
                              "gonenoaddress":"parti sans laisser d'adresse",
                              "lost":"carte perdue",
                              "debarred":"date de suspension",
                              "contactname":"nom du garant",
                              "contactfirstname":"prénom du garant",
                              "relationship":"relation garant",
                              "password":"mot de passe",
                              "contactnote":"documents à payer",
                              "altcontactfirstname":"lieu et date de naissance de l'adhérent",
                              "altcontactsurname":"lieu de naissance du garant"})

In [41]:
anomalies_villes = adherents

In [42]:
villes_frontalieres_rbx = ["CROIX", "HEM", "LEERS", "LYS-LEZ-LANNOY", "ROUBAIX", "TOURCOING", "WATTRELOS", "TOMBOUCTOU"]
for ville in villes_frontalieres_rbx:
    anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match(ville)]

In [43]:
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VILLENEUVE-D'ASCQ")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MARCQ-EN-BAROEUL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LAMBERSART")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ARMENTIERES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LOOS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LA MADELEINE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MONS-EN-BAROEUL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WASQUEHAL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HALLUIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("RONCHIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("FACHES-THUMESNIL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WATTIGNIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HAUBOURDIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("RONCQ")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MOUVAUX")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-ANDRE-LEZ-LILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("COMINES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SECLIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MARQUETTE-LEZ-LILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WAMBRECHIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ANNOEULLIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NEUVILLE-EN-FERRAIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BONDUES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LEERS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LESQUIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LA CHAPELLE-D'ARMENTIERES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("PERENCHIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LINSELLES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HOUPLINES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WAVRIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("QUESNOY-SUR-DEULE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LA BRASSEE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SANTES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINHGIN-EN-WEPPES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ERQUINGHEM-LYS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WERVICQ-SUD")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BAUVIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BAISIEUX")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SEQUEDIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BOUSBECQUE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HALLENNES-LEZ-HAUBOURDIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("PROVIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("TOUFFLERS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ALLENNES-LES-MARAIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("TEMPLEMARS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HOUPLIN-ANCOISNE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("FRETIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("EMMERIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LEZENNES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WILLEMS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SALOME")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CHERENG")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINGHIN-EN-MELANTOIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VERLINGHEM")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CAPINGHEM")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("FRELINGHIEN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HERLIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("FOURNES-EN-WEPPES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LOMPRET")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("PREMESQUES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MARQUILLIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAILLY-LEZ-LANNOY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("DEULEMONT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LANNOY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BOIS-GRENIER")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AUBERS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ILLIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ANSTAING")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VENDEVILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("FOREST-SUR-MARQUE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("TRESSIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("RADINGHEM-EN-WEPPES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("DON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ENNETIERES-EN-WEPPES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HANTAY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GRUSON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("FROMELLES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CARNIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("PERONNE-EN-MELANTOIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NOYELLE-LES-SECLIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BEAUCAMPS-LIGNY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BOUVINES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LE MAISNIL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ENGLOS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ERQUINGHEM-LE-SEC")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WICRES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ESCOBECQUES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WARNETON")]

In [44]:
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ANZIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ATTICHES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AUBY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AULNOY-LEZ-VALENCIENNES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AVELIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AVESNES-SUR-HELPE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BACHY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BAILLEUL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BERGUES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BOUCHAIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BOURGHELLES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BOUSIGNIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CAESTRE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CAMPHIN-EN-PEVELE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CAPPELLE-BROUCK")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CAPPELLE-LA-GRANDE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CLAMART")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("COURCHELETTES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CYSOING")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("DENAIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("DOUAI")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("DUNKERQUE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ERRE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("FELLERIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GENECH")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GONDECOURT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GRAVELINES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HASNON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HAUTMONT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HAZEBROUCK")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HELLEMMES-LILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HERGNIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LA BASSEE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LAMBVRE-LEZ-DOUAI")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LANDRECIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LAUWIN-PLANQUE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LE PLESSIS-ROBINSON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LOMME")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LOON-PLAGE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LOUVROIL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MAUBEUGE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MERRIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MERVILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MONS-EN-PEVELE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MORTAGNE-DU-NORD")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NIEPPE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NOYELLES-LES-SECLIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("OHAIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ONNAING")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ORCHIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("OSTRICOURT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("PHALEMPIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("PONT-A-MARCQ")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("RAISMES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINGHIN-EN-WEPPES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-AMAND-LES-EAUX")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-ANDRE-LEZ-LILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-SAULVE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-VAAST-EN-CAMBRESIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAMEON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SIN-LE-NOBLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SOMAIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("STEENVOORDE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("STEENWERCK")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("TEMPLEUVE-EN-PEVELE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("TERDEGHEM")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("THUMERIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VALENCIENNES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VIEUX CONDE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WATTRELOS")]

In [45]:
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AIX-NOULETTE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ANGRES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ARRAS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ARQUES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AUCHEL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AUDIGHEN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BAINCTHUN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BLANGERVAL-BLANGERMONT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BRUAY-LA-BUISSIERE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CALAIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CARVIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("COURCELLES-LES-LENS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CREQUY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("DOUVRIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HAISNES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HESDIGNEUL-LES-BETHUNE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HINGES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ISBERGUES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LA COMTE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LIBERCOURT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LIEVIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MONTIGNY-EN-GOHELLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NOEUX-LES-MINES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("OIGNIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("OUTREAU")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("PONT-A-VENDIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ROUVROY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SALLAUMINES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAMER")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SERICOURT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VAUDRICOURT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VENDIN-LE-VIEIL")]

In [46]:
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BRUXELLES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CHARLEROI")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CHAUMONT-GISTOUX")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("DOTTIGNIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ESTAIMBOURG")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ESTAIMPUIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GAND")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GINGELOM")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HERINNES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HERSEAUX")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LEUZE-EN-HAINAUT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LIEGE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MENIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MONS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MOUSCRON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NECHIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("OBIGIES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("RAMEGNIES-CHIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("REKKEM")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("TEMPLEUVE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("TOURNAI")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("WARCOING")]

In [47]:
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AIZENAY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ANGERS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ANTIBES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("AUBIGNY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BAGNOLET")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BOISSY-MAUVOISIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BORDEAUX")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BORRE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("BREST")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CAEN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CANNES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CARRIERES-SOUS-POISSY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CHERBOURG")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("COMBS-LA-VILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("CONDRIEU")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("DIJON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ERMENONVILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("EVREUX")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("EYBENS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("FLERS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GRENOBLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GANZEVILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("GENNES-SUR-SEICHE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("IVRY-SUR-SEINE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("JUAN-LES-PINS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LACROIX-SAINT-OUEN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LA GARNACHE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LAPTE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LE MANS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LE NOUVION-EN-THIERACHE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LYON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MAGNANVILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MARLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MARSEILLE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MAULEON-LICHARRE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MAZAMET")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MONTFORT-L'AMAURY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NANCY")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NIBAS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NOGENTEL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("NOISY-LE-GRAND")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("OGNOLLES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("OULLINS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("PARIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("LES PAVILLONS-SOUS-BOIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("RENNES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("ROUEN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("RUEIL-MALMAISON")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAILLY-FLIBEAUCOURT")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-DENIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-JEAN-KERDANIEL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-LOUIS")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-MARTIN-LE-NOEUD")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-MAUR-DES-FOSSES")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAINT-ROMAIN-EN-GAL")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAULCE-SUR-RHONE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SAULCE-MONCLIN")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("SEGRIE-FONTAINE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("STRASBOURG")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("THOUROTE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("TOULOUSE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VALENCE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VIC-LE-COMTE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VIENNE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VILLEPINTE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VILLEURBANNE")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("VOIRON")]

In [48]:
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("HUELVA")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MADRID")]
anomalies_villes = anomalies_villes[~anomalies_villes["ville"].str.match("MONCHENGLADBACH")]

In [49]:
anomalies_villes.groupby("ville").size()

ville
13 ALLÉE DES FRÈRES DILLIES    1
170, RUE JEAN JAURÈS           1
206 RUE DE TOURCOING           1
33000                          1
39 RUE DU DOCTEUR CASTIAU      1
46 RUE JOSEPH                  1
49 RUE OSWALD CRESPI           1
53 BOULEVARD VAN GOGH          1
59280                          1
8 RUE DORÉE                    1
86 RUE CLAUDE JEAN             1
98 BOULEVARD CLÉMENÇEAU        1
BRUAY LA BUYSSIERE             1
CLERMONT-FERRAND               1
COUCELLES-LES-LENS             1
EVREGNIES                      2
FLETRE                         1
GONNEVILLE                     1
LAMBVRES-LEZ-DOUAI             1
LEMPDES                        1
LENS                           1
LES PAVILLONS SOUS BOIS        1
MONT-DE-MARSAN                 1
RUMES                          1
SAINT-GERMAIN-SOUS-CAILLY      1
VILLABE                        1
WAHAGNIES                      1
dtype: int64

In [19]:
anomalies_villes[['userid', 'ville', 'code postal']].to_excel('villes_ko.xlsx', index=False)

In [20]:
villes_corr = pd.read_excel("villes_ko_corr.xlsx")

In [21]:
villes_corr

,Unnamed: 0,userid,ville,ville_corr
0,4051,ROXANE608,11 AVENUE DES DEUX MOULINS,CROIX
1,9040,X0002369372,13 RUE MIRABEAU,CROIX
2,2268,X0002464060,"13, RUE CHARLES GOUNOD",CROIX
3,10812,X0002279534,14 AVENUE EDOUARD VAILLANT,CROIX
4,2163,X0002380506,14/14 RUE EUGÈNE GUILLAUME,CROIX
...,...,...,...,...
154,22969,X0002557144,10 RUE DES FRÈRES HOLLEBCEQUE,WERVICQ-SUD
155,12635,X0002361093,29 ALLÉE DES ROSES,WERVICQ-SUD
156,25198,X0002584744,8 LES JARDINS DE LYS,WERVICQ-SUD
157,5367,X0002154008,WERVICQ SUD,WERVICQ-SUD


In [22]:
villes_corr = villes_corr[['userid', 'ville_corr']]

In [23]:
villes_corr

,userid,ville_corr
0,ROXANE608,CROIX
1,X0002369372,CROIX
2,X0002464060,CROIX
3,X0002279534,CROIX
4,X0002380506,CROIX
...,...,...
154,X0002557144,WERVICQ-SUD
155,X0002361093,WERVICQ-SUD
156,X0002584744,WERVICQ-SUD
157,X0002154008,WERVICQ-SUD


In [35]:
test = villes_corr[3:4]
test

,userid,ville_corr
3,X0002279534,CROIX


In [ ]:
#for b in rbx_adr_ko_cleaned[rbx_adr_ko_cleaned['statut'] == True].to_dict(orient='records'):
#    data = {"address": b["clean_address"], "state": b["id_cicn2"], "altcontactcountry": b["irisInsee"]}
#    mod_borrower(b["userid"], data)

In [37]:
for b in villes_corr.to_dict(orient='records'):
    data = {"city": b["ville_corr"]}
    mod_borrower(b["userid"], data)

data={"city": "CROIX"}
ROXANE608 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "CROIX"}
X0002369372 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "CROIX"}
X0002464060 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "CROIX"}
X0002279534 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "CROIX"}
X0002380506 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "CROIX"}
X0002545011 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "CROIX"}
X0002553252 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "CROIX"}
X0002545059 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "CROIX"}
X0002553245 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "CROIX"}
X0002093093 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "CROIX"}
X0002466354 : b'{\n

X0002600598 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "ROUBAIX"}
X0002600604 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "ROUBAIX"}
X0002595054 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "ROUBAIX"}
X0002061177 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "ROUBAIX"}
X0002602950 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "ROUBAIX"}
X0002468709 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "ROUBAIX"}
X0002591247 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "ROUBAIX"}
X0002602653 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "ROUBAIX"}
X0002604275 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
data={"city": "ROUBAIX"}
X0002605340 : b'{\n   "success" : true,\n   "modified_fields" : {}\n}\n'
data={"city": "ROUBAIX"}
X0002066936 : b'{\n 